# Liquidity Pools

They keep the product constant and adjust the price(s) according to demand / supply.

In [19]:
MRCT_Q = 50000
ETH_Q = 50000

K = MRCT_Q * ETH_Q

mrct = float(MRCT_Q)
eth = float(ETH_Q)

mrct_p = 1.0
eth_p = 1.0

def printp():
    print(f"Pool :: MRCT ${mrct_p:.6} ({mrct:.6}), ETH ${eth_p:.6} ({eth:.6})")


After setting the initial pool to have a constant value of 2.5BN (50k x 50k) we need to keep this balance.

Assume someone wishes to BUY 1,000 MRCT tokens:

In [20]:
mrct -= 1000
eth = K / mrct

mrct_p = MRCT_Q / mrct
eth_p = ETH_Q / eth

printp()

Pool :: MRCT $1.02041 (49000.0), ETH $0.98 (51020.4)


Now someone sells into the pool 2,500 ETH:

In [21]:
eth += 2000

mrct = K / eth

mrct_p = MRCT_Q / mrct
eth_p = ETH_Q / eth

printp()

Pool :: MRCT $1.06041 (47151.7), ETH $0.943033 (53020.4)


In [25]:
# Pool value stays constant:
print(f"${eth_p * eth * mrct_p * mrct / 10**9:.2}BN")

$2.5BN


So, we arrive at the trading algo, where `sym` is the asset we want to trade (`ETH` or `MRCT`) and `qty` is how much of it want to buy from (negative) or sell into (positive) the pool:

In [56]:
class Pool:
    ETH = 'ETH'
    MRCT = 'MRCT'
    
    def __init__(self, qty):
        self.Q = qty
        self.eth = self.mrct = self.Q
        self.K = self.Q * self.Q
        self.mrct_p = 1.0
        self.eth_p = 1.0

    def __repr__(self):
        return f"Pool :: MRCT ${self.mrct_p:.6f} ({self.mrct:.6f}), ETH ${self.eth_p:.6f} ({self.eth:.6f})"

    def trade(self, sym: str, qty: float):
        value = 0
        if sym == Pool.ETH:
            self.eth += qty
            desired = self.K / self.eth
            diff = self.mrct - desired
            if diff > 0:
                print(f"Emitting {diff:.5f} MRCTs")
            else:
                print(f"Requesting {-diff:.5f} MRCTs")
            self.mrct = desired
            
        elif sym == Pool.MRCT:
            self.mrct += qty
            desired = self.K / self.mrct
            diff = self.eth - desired
            if diff > 0:
                print(f"Emitting {diff:.5f} ETH")
            else:
                print(f"Requesting {-diff:.5f} ETH")
            self.eth = desired
            
        else:
            raise RuntimeError(f"Not a valid security: {sym}")

        self.mrct_p = self.Q / self.mrct
        self.eth_p = self.Q / self.eth
        return diff


In [57]:
# Build a Pool with 30,000 each of ETH and MRCT:
pool = Pool(30000)

# We are "pumping" MRCT by flooding the pool with ETH
mrct = 0
mrct += pool.trade('ETH', 1000)
mrct += pool.trade('ETH', 2000)
mrct += pool.trade('ETH', 10000)

print(f"{pool}")
print(f"We now have {mrct:.4f} MRCT, for a total value of ${mrct * pool.mrct_p:.4f}")

Emitting 967.74194 MRCTs
Emitting 1759.53079 MRCTs
Emitting 6342.49471 MRCTs
Pool :: MRCT $1.433333 (20930.232558), ETH $0.697674 (43000.000000)
We now have 9069.7674 MRCT, for a total value of $13000.0000


The ETH we used to flood the pool was bought (assuming parity) for USD13,000 so no gain was had, but if we had originally 10,000 MRCTs (worth $10,000) they are now worth:

In [58]:
10000 * pool.mrct_p

14333.333333333334

so we have made a "gain" of around 43%, which we can try to "realize":

In [59]:
eth = pool.trade('MRCT', 10000)
print(f"{pool}")
print(f"We now have {eth:.4f} ETH, and {mrct:.4f} MRCT")
print(f"Their value is: ${eth*pool.eth_p + mrct*pool.mrct_p:.4f}")

Emitting 13902.25564 ETH
Pool :: MRCT $0.969925 (30930.232558), ETH $1.031008 (29097.744361)
We now have 13902.2556 ETH, and 9069.7674 MRCT
Their value is: $23130.3258
